In [55]:
import pandas as pd
import os
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import RidgeClassifier
import numpy as np
from statsmodels.discrete.discrete_model import Logit
os.chdir('/data/arpit.goel/20_TwoSigmaFinancial')

train = pd.read_pickle('01.RawData/train_split.pkl').sample(n=100000)
test_ts = pd.read_pickle('01.RawData/test_split_ts.pkl')
test_id = pd.read_pickle('01.RawData/test_split_id.pkl')

tagging=pd.read_csv('03.ProcessedData/02.Tagging_id.csv')
train=pd.merge(train,tagging,on='id')
test_ts=pd.merge(test_ts,tagging,on='id')
test_id=pd.merge(test_id,tagging,on='id')

msk = np.random.rand(len(train)) < 0.6
ins = train[msk]
oos = train[~msk]



In [57]:
cols = [x for x in train.columns.tolist() if x not in ['y','timestamp','id','group']]

cols=['technical_7','technical_40','fundamental_39','fundamental_8','fundamental_14','fundamental_5',\
      'fundamental_40','fundamental_21','fundamental_58','fundamental_51','fundamental_0','fundamental_22',\
      'fundamental_46','fundamental_13','fundamental_16','fundamental_57','fundamental_23','derived_3',\
      'fundamental_61','fundamental_10','fundamental_24','fundamental_9','fundamental_12','fundamental_15',\
      'fundamental_59']


n_estimators=20
gb = GradientBoostingClassifier(n_estimators=n_estimators, max_depth=3)
#gb = RidgeClassifier()
gb.fit(ins[cols],ins['group'])
print n_estimators, gb.score(ins[cols],ins['group']), gb.score(oos[cols],oos['group'])
print gb.score(test_ts[cols],test_ts['group']), gb.score(test_id[cols],test_id['group'])

gb = RidgeClassifier()
gb.fit(ins[cols],ins['group'])
print n_estimators, gb.score(ins[cols],ins['group']), gb.score(oos[cols],oos['group'])
print gb.score(test_ts[cols],test_ts['group']), gb.score(test_id[cols],test_id['group'])



20 0.750050033356 0.746928071928
0.637694053563 0.667784342197
20 0.636757838559 0.640534465534
0.541470721743 0.574808517242


In [34]:
pd.DataFrame(gb.predict_proba(ins[cols].head()),columns=['p%d'%x for x in range(6)])



,p0,p1,p2,p3,p4,p5
0,0.544625,0.044764,0.136031,0.039569,0.028187,0.206825
1,0.534582,0.044920,0.148155,0.044511,0.030412,0.197420
2,0.571649,0.042967,0.115321,0.050947,0.030280,0.188837
3,0.268840,0.108231,0.222561,0.146529,0.057912,0.195927
4,0.207727,0.115818,0.100478,0.155896,0.047586,0.372496


In [46]:
#feature_importance=pd.DataFrame(zip(cols,list(gb.feature_importances_)),columns=['variable','importance']).sort_values(by=['importance'],ascending=False)
#feature_importance['Cumm']=feature_importance['importance'].cumsum()
feature_importance=pd.DataFrame(gb.coef_,columns=cols).T
feature_importance.to_csv('03.ProcessedData/RidgeLRPredictIDClassWeights.csv')

array([[ -7.37186249e-03,   9.75012845e-02,  -6.07326876e-03,
         -2.80855311e-02,   5.85774041e-03,  -1.99196878e-02,
         -7.89969194e-02,   1.66240257e-02,   1.20614497e-01,
          2.15689874e-01,  -1.83124206e-02,  -7.42052032e-02,
         -2.01932780e-01,  -3.05765300e-01,   4.77198027e-02,
          5.65301221e-02,  -9.56492386e-03,   9.89076298e-02,
          5.73606694e-02,   1.58652391e-01,   2.43814973e-01,
          1.02221748e-02,  -6.31656965e-02,   2.48926166e-02,
         -6.03905143e-01,   1.05013778e-01,  -5.73498238e-02,
         -1.24187498e-01,   7.62460141e-02,   2.77667922e-01,
          4.32688206e-02,  -2.79533109e-02,  -1.90968358e-01,
          1.01664600e-01,  -7.37434220e-02,  -3.40465959e-02,
         -6.45815528e-03,  -2.61963508e-02,   5.24499294e-03,
         -1.11231730e-03,  -2.61458188e-02,   1.09597214e-01,
          2.45110043e-02,  -1.77461518e-01,  -5.22579971e-01,
          1.45128611e-02,  -6.01373137e-03,   1.60375734e-02,
        

In [64]:
missing_values=[-0.008, 0.065, 0.031, 0.04, 0.027, -0.026, 0.004, -0.043, 0.007, 0.156, 0.038, 0.046, 0.02, 0.051, 0.1, -0.086, 0.127, 0.052, 0.083, 0.073,\
            0.003, -0.104, -0.054, 0.106, 0.089, 0.08, 0.004, 0.144, 0.091, 0.012, 0.046, 0.055, 0.013, 0.08, 0.06, 0.018, 0.08, 0.055, 0.075, 0.064, -0.032,\
            0.065, 0.025, 0.042, -0.022, -0.064, -0.078, 0.121, 0.103, -0.092, 0.089, 0.034, -0.043, 0.106, 0.154, 0.07, 0.052, -0.026, 0.154, -0.026, 0.017,\
            0.065, 0, 0.115, 0.077, 0.074, -0.064, 0.126, -0.087, 0, -0.909, 0.001, 0, -0.988, 0.018, 0, -0.723, -0.945, -0.17, 0, -1, 0, -0.895, 0, -0.097,\
            0.001, -0.031, -0.002, 0.001, 0.001, -0.125, 0.001, -0.857, 0.001, 0, -0.043, 0.012, 0.001, -0.123, -0.125, -0.054, -0.034, -0.022, 0.024, 0.004,\
            0, -1.008, 0, 0]

In [37]:
import pandas as pd
import os
from sklearn.ensemble import GradientBoostingClassifier
import numpy as np

os.chdir('/data/arpit.goel/20_TwoSigmaFinancial')

train = pd.read_pickle('01.RawData/train_split.pkl').sample(n=100000)
test_ts = pd.read_pickle('01.RawData/test_split_ts.pkl')
test_id = pd.read_pickle('01.RawData/test_split_id.pkl')

tagging=pd.read_csv('03.ProcessedData/02.Tagging_timestamp.csv')
train=pd.merge(train,tagging,on='timestamp')
test_ts=pd.merge(test_ts,tagging,on='timestamp')
test_id=pd.merge(test_id,tagging,on='timestamp')

msk = np.random.rand(len(train)) < 0.6
ins = train[msk]
oos = train[~msk]



In [38]:
cols = [x for x in train.columns.tolist() if x not in ['y','timestamp','id','group']]

n_estimators=20
gb = GradientBoostingClassifier(n_estimators=n_estimators, max_depth=3)
gb.fit(ins[cols],ins['group'])
print n_estimators, gb.score(ins[cols],ins['group']), gb.score(oos[cols],oos['group'])
#print gb.score(test_ts[cols],test_ts['group']), gb.score(test_id[cols],test_id['group'])



20 0.210579109268 0.179879797501


In [39]:
feature_importance=pd.DataFrame(zip(cols,list(gb.feature_importances_)),columns=['variable','importance']).sort_values(by=['importance'],ascending=False)
feature_importance['Cumm']=feature_importance['importance'].cumsum()
feature_importance

,variable,importance,Cumm
91,technical_28,8.879792e-02,0.088798
103,technical_40,7.573760e-02,0.164536
70,technical_2,5.773730e-02,0.222273
77,technical_11,5.265232e-02,0.274925
73,technical_6,3.590311e-02,0.310828
82,technical_17,3.392493e-02,0.344753
95,technical_32,3.129667e-02,0.376050
74,technical_7,2.989305e-02,0.405943
93,technical_30,2.503562e-02,0.430979
84,technical_19,2.433226e-02,0.455311
